# 9/6 Reading B&O 8.0 - 8.1

### 8.0 Exceptional Control Flow

From the moment the computer is turned on, the PC assumes a sequence of adresses from a0 to an that points to an instruction. Each move from ak to ak+1 is called a control transfer. A sequence of these moves is called the control flow of a processor.

The simplest kind of control flow is a **smooth sequence** where the addresses are right next to each other. Changes to the control flow occur due to jumps, calls, and returns. These mechanisms are necessary to allow the program to react to changes in the internal program state represented by program variables. There are also lots of other changes in system state that must be handled, such as network packets, hardware timers, etc. etc.

Modern systems react to these changes by making abrupt changes to the control flow, these changes are called exceptional control flows (ECF).

Understanding ECF is important in that it will help you understance important system concepts, how applications interact with the operating system, write interesting application programs, understand concurrency, how software exceptions work, and many others.

## 8.1 Exceptions

Exceptions are a form of ECF partially implemented by the OS and hardware, so the details are different from system to system, but the idea is the same.  
An exception is an abrupt change in resopnse to some change in the processor state.  
For example, the processor is executing the current instruction when a significant change in the processor state occurs.  
This state is encoded in various bits and signals inside the processor. The change is called an event. The event might be related to the execution of the current instruction. When the processor detects that an event has occurred, it makes an indirect procedural call (the exception) through a jump table called the exception table, to an OS subroutine (the exception handler) that is designed to process this event. When the exception handler finishes processing, one of three things could happen.   
1. The handler returns control to the current instruction.  
2. The handler returns control to the next instruction, which would have been executed had the exception not occurred.  
3. The handler aborts the program

### 8.1.1 Exception Handling

Each type of exception is assigned a unique nonnegative exception number. Some of these number are assigned by designers of the processor, and others are assigned by the designers of the OS kernel.   
At system boot, the OS allocates and initializes an exception table.  
At runtime, the processor detects that an event has occurred and determines the exception number. The processor then triggers the exception by making a call to exception number in the exception table. The exception number is an index in the exception table, whose starting address is located in a CPU register called the exception table base register. 

An exception is similar to a procedure call, but with several differences:   
1. Just like any other procedural call, the processor pushes a return address on the stack, but this can either be the current instruction or the next instruction.  
2. The processor also pushes the processor state onto the stack that will be necessary to restart the interrupted program.  
3. When control is being transferred from a user program to the kernel, all the items are pushed onto the kernel stack rather than the user stack.  
4. Exception handlers run in kernel mode, which means they have complete access to all system resources.

Once the handler triggers the exception, the rest of the work is done in software by the exception handler. After the handler has processed the event, it can return to the program by executing a return from interrupt instruction, which restores the state to the user mode (returns the state prior to the exception to the machine registers, etc) if the exception interrupted a user program and then returns control to the interrupted program.

### 8.1.2 Classes of Exceptions

Exceptions can be divided into four classes: interrupts, traps, faults, and aborts.

#### Interrupts

**Cause: Signal from I/O device  
Return Behavior: Always return to next instruction**  
Interrupts occur asynchronosly as a result of signals from I/O devices that are external to the processor. Hardware interrupts are asynchronous (not caused by the execution of any particular instruction). Exception handlers for hardware interrupts are called interrupt handlers.    
After the current instruction finishes executing, the processor notices that the interrupt pin has gone high, reads the exception number, and then calls the interrupt handler. When the handler returns, control is returned to the next instruction. The effect is that the program continues executing as if the interrupt had not happened. The rest of the exceptions are synchronous due to the current instruction. This instruction is also called the faulting instruction.

#### Trap and System Calls

**Cause: Intentional Exception   
Return Behavior: Always returns to the next instruction**  
Traps are intentional exception that occur as a result of an instruction exception. The most important use of a trap is to provide an interface between user programs and the kernel. This is called a **system call**.   
User programs often need to request services from the kernel. To allow controlled access to kernel services, processors provide syscall instructions for a specific service that user programs can execute. Executing the syscall instruction causes a trap that decodes the argument and calls the appropriate kernel routine.

Syscalls may seem similar to functions, but functions run in user mode, which restricts the type of instructions that can be executed, but a syscall runs in kernel mode which allows priveleged access.

#### Faults

**Cause: Potentially recoverable error   
Return behavior: Might return to current instruction**  
Faults result from error conditions the handler might be able to correct. When a fault occurs, the processor transfers control to the fault handler, and if the handler is able to correct the error condition, it returns control to the faulting instruction, reexecuting it. Otherwise, the handler aborts, terminating the application program.

#### Aborts

**Cause: Nonrecoverable error   
Return behavior: Never returns**  
Aborts result from unrecoverable fatal error, typically when DRAM or SRAM bits are corrupted. Aborts cause an abort routine that terminate an application program.  

### 8.1.3 Exceptions in Linux/x86-64 systems

Faults and aborts: 
1. Divide error (divide by zero error or too big, normally floating exceptions)  
2. General Protection fault: occurs usually because program references an undefined area of memory or if the program references an undefined area of virtual memory or because the program writes to a read only segment. Linux reports this as a seg fault   
3. Page fault: Exception where the faulting instruction is restarted. Handler maps a page of virtual memory on disk into a page of physical memory and restarts the instruction   
4. Machine check: fatal hardware error that is detected during execution of faulting instruction, never returns to program


Syscalls: 
C programs can invoke system calls using the syscall function. The c standard library provides wrapper functions for most systemcalls. These wrapper functions package the arguments, trap the kernel with the system call and then passes the return status of the system call back to the calling program. These wrapper functions will be referred to as suystem level functions.